In [71]:
import pandas as pd
import numpy as np

In [53]:
path = "/neurospin/dico/data/deep_folding/current/datasets/UkBioBank"
brain_volume_file = f"{path}/brain_segmentation_volume.csv"
participants_file = f"{path}/participants_sex_age.csv"
participants_502356_subjects_file = f"{path}/participants_sex_age_site_502356-subjects.csv"
target_file = f"{path}/participants.csv"

In [54]:
volume = pd.read_csv(brain_volume_file, header=None).rename(columns={0: "participant_id", 1: "BrainVolumeFreeSurfer_mm3"}).set_index("participant_id")
volume

,BrainVolumeFreeSurfer_mm3
participant_id,
sub-1000021,1310943.0
sub-1000325,1312205.0
sub-1000458,1377807.0
sub-1000575,1111215.0
sub-1000606,1229380.0
...,...
sub-6023808,1117542.0
sub-6023847,991646.0
sub-6024150,1150009.0


In [55]:
participants = pd.read_csv(participants_file, index_col=0)
participants

,Sex,Age
participant_id,,
sub-3411469,1,65.0
sub-2290292,0,69.0
sub-3842391,0,70.0
sub-2758537,1,49.0
sub-4334619,1,70.0
...,...,...
sub-5057890,1,72.0
sub-2038574,1,70.0
sub-1415693,0,64.0


In [56]:
all_participants = pd.read_csv(participants_502356_subjects_file).set_index("participant_id")
all_participants = all_participants.set_index('sub-' + all_participants.index.astype(str))
all_participants

,Sex,Ageattendedimaging,ImagingName
participant_id,,,
sub-1000016,1,NaN,NaN
sub-1000021,1,51.0,Cheadle
sub-1000033,1,NaN,NaN
sub-1000049,0,NaN,NaN
sub-1000057,1,NaN,NaN
...,...,...,...
sub-6024912,1,NaN,NaN
sub-6024923,0,NaN,NaN
sub-6024930,0,NaN,NaN


In [57]:
target = pd.merge(participants, volume, left_index=True, right_index=True)
target = pd.merge(target, all_participants.iloc[:, 1:], left_index=True, right_index=True)
target

,Sex,Age,BrainVolumeFreeSurfer_mm3,Ageattendedimaging,ImagingName
participant_id,,,,,
sub-3411469,1,65.0,1136465.0,65.0,Cheadle
sub-2290292,0,69.0,1079118.0,69.0,Newcastle
sub-3842391,0,70.0,1124077.0,70.0,Cheadle
sub-2758537,1,49.0,1246758.0,49.0,Newcastle
sub-4334619,1,70.0,1410387.0,70.0,Cheadle
...,...,...,...,...,...
sub-2621329,1,56.0,1260788.0,56.0,Cheadle
sub-5057890,1,72.0,1328760.0,72.0,Cheadle
sub-2038574,1,70.0,1244624.0,70.0,Reading


In [58]:
q_10 = target["BrainVolumeFreeSurfer_mm3"].quantile(q=0.1)
q_90 = target["BrainVolumeFreeSurfer_mm3"].quantile(q=0.9)

In [78]:
target["isBigBrain"] = target.apply(lambda x: 0 if x['BrainVolumeFreeSurfer_mm3'] < q_10 else 1 if x['BrainVolumeFreeSurfer_mm3'] > q_90 else np.nan, axis=1)
target["isBigBrain"] = target["isBigBrain"].astype('Int64')
target

,Sex,Age,BrainVolumeFreeSurfer_mm3,Ageattendedimaging,ImagingName,isBigBrain
participant_id,,,,,,
sub-3411469,1,65.0,1136465.0,65.0,Cheadle,<NA>
sub-2290292,0,69.0,1079118.0,69.0,Newcastle,<NA>
sub-3842391,0,70.0,1124077.0,70.0,Cheadle,<NA>
sub-2758537,1,49.0,1246758.0,49.0,Newcastle,<NA>
sub-4334619,1,70.0,1410387.0,70.0,Cheadle,1
...,...,...,...,...,...,...
sub-2621329,1,56.0,1260788.0,56.0,Cheadle,<NA>
sub-5057890,1,72.0,1328760.0,72.0,Cheadle,<NA>
sub-2038574,1,70.0,1244624.0,70.0,Reading,<NA>


In [79]:
target.dtypes

Sex                            int64
Age                          float64
BrainVolumeFreeSurfer_mm3    float64
Ageattendedimaging           float64
ImagingName                   object
isBigBrain                     Int64
dtype: object

In [80]:
target.to_csv(target_file)